In [1]:
%reload_ext autoreload
%autoreload 2

%gui qt

import time
from tqdm import tqdm
from pathlib import Path
import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import fileManagement as fm
import redCellGUI as rgui
import dbManagement as dbm

In [3]:
# Immediate next steps for programming work:
# - ROICaT index alignment tools

In [113]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'
# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In session: CR_Hippocannula6/2022-09-05/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Convert mpciROIs.planeIndex to mpciROIs.stackPosition, with [X,Y,Z(planeIndex)] coordinate for each ROI!
Computing red cell features... (usually takes 10-20 seconds)
C:\Users\andrew\Documents\localData\CR_Hippocannula6\2022-09-05\701


In [30]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [3]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In [3]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [13]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 3.58 s
Wall time: 512 ms


In [147]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [30]:
# fm.getCopyString('ATL027')
fm.s2pTargets('ATL012','2023-08-08','701')

\\zaru.cortexlab.net\Subjects\ATL028\2023-08-08\701
C:\Users\andrew\Documents\localData\ATL028\2023-08-08\701


In [35]:
# Database Ideas: 
# ---------------------
# GUI: db manager
# - show full database -
# - show selected entries (like for example entries with imaging but no suite2p) -
#                --> It would be sweet to make this work where there are some defaults, but then there is also a "filter command" option which you type and goes as the input to the filtering
# - click on entry and do things:
#                --> open file explorer to that session
#                --> do suite2p
#                --> do red cell management
# - update table data? 
# ---------------------
# Operational Commands: 
# - Automatically do suite2p 
# - ??? 

In [3]:
vrdb = dbm.vrDatabase()

In [75]:
vrdb.tableData()[0]

['uSessionID',
 'mouseName',
 'sessionDate',
 'sessionID',
 'experimentType',
 'experimentID',
 'variableGain',
 'behavior',
 'imaging',
 'faceCamera',
 'vrEnvironments',
 'headPlateRotation',
 'numPlanes',
 'planeSeparation',
 'pockelsPercentage',
 'objectiveRotation',
 'vrRegistration',
 'suite2p',
 'suite2pQC',
 'redCellQC',
 'sessionQC',
 'scratchJustification',
 'logtime',
 'sessionNotes']

In [36]:
df = vrdb.showTable()

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [38]:
vrdb.checkS2P()

In [21]:
vrdb.printRequiresS2P()

In [39]:
# Before registering everything, do this: 
print("Convert mpciROIs.planeIndex to mpciROIs.stackPosition, with [X,Y,Z(planeIndex)] coordinate for each ROI!")

Convert mpciROIs.planeIndex to mpciROIs.stackPosition, with [X,Y,Z(planeIndex)] coordinate for each ROI!


In [ ]:
xx = vrdb.getTable(ignoreScratched=False, imaging=True, **{"sessionQC":False})
vrdb.showTable(xx)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [73]:
def tableData(self):
    with self.openCursor() as cursor:
        fieldNames = [col.column_name for col in cursor.columns(table=self.tableName)]
        cursor.execute(f"SELECT * FROM {self.tableName}")
        tableElements = cursor.fetchall()
    return tableElements, fieldNames

te, fn = tableData(vrdb)

In [74]:
te

[(1, 'ATL012', datetime.datetime(2023, 1, 18, 0, 0), 701, 'Blender VR', 1, False, True, True, False, 1, None, 5, 25.0, 6.0, 6.300000190734863, False, True, False, False, True, None, None, 'Very few trials, mouse not trained (for about 6 days) and had it on the wheel an extra 30 minutes...'),
 (2, 'ATL012', datetime.datetime(2023, 1, 20, 0, 0), 701, 'Blender VR', 1, False, True, True, False, 1, 328.0, 5, 25.0, 8.0, 14.0, False, True, False, False, True, None, None, 'Decent number of trials considering no reward :('),
 (3, 'ATL012', datetime.datetime(2023, 1, 20, 0, 0), 702, 'Blender VR', 1, False, True, True, False, 1, 328.0, 5, 25.0, 8.0, 14.0, False, True, False, False, True, None, None, 'Got a surprising number of trials out of it now that reward is working again'),
 (4, 'ATL012', datetime.datetime(2023, 1, 23, 0, 0), 701, 'Blender VR', 1, False, True, True, False, 1, 328.0, 5, 25.0, 6.0, 14.0, False, True, False, False, True, None, None, 'Mouse sending it, but water was almost dry i